<a href="https://colab.research.google.com/github/satishsinghtoo/Deep-Leraning/blob/main/OptimizeCNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create CNN Model and Optimize it using Keras Tuner

In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.8 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.15.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-10-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 32s]
val_accuracy: 0.8510000109672546

Best val_accuracy So Far: 0.9104999899864197
Total elapsed time: 00h 03m 14s


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 96)        960       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        115248    
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 112)               2602096   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 2719434 (10.37 MB)
Trainable params: 2719434 (10.37 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.1353 - accuracy: 0.9494 - val_loss: 0.2588 - val_accuracy: 0.9122
Epoch 5/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0956 - accuracy: 0.9645 - val_loss: 0.3013 - val_accuracy: 0.9102
Epoch 6/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0662 - accuracy: 0.9756 - val_loss: 0.3436 - val_accuracy: 0.9048
Epoch 7/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0488 - accuracy: 0.9822 - val_loss: 0.3888 - val_accuracy: 0.9133
Epoch 8/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0368 - accuracy: 0.9866 - val_loss: 0.4407 - val_accuracy: 0.9087
Epoch 9/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0315 - accuracy: 0.9891 - val_loss: 0.4739 - val_accuracy: 0.9090
Epoch 10/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0262 - accuracy: 0.9909 - val_loss: 0.5838 - val_a